# Deploy and Query Models in Databricks

Thursday, Jun 12, 2025

[Invitation on Luma](https://lu.ma/avdu1fev), [LinkedIn](https://www.linkedin.com/feed/update/urn:li:activity:7337118509803393026), [Meetup](https://www.meetup.com/warsaw-data-engineering/events/308350480/)


# 📚 Agenda

1. **10 minut** Ogłoszenia. Czas na szalone pomysły na przyszłe spotkania 🫣👻
1. **55 minut** Demo Time 🧑‍💻
    1. Demo: Model Deployment (Serving, Publishing) in Databricks
1. **10 minut** Pytania i odpowiedzi 🙋‍♀️

⏰ Całkowity czas trwania spotkania: **1h 15min**


# 📈 LinkedIn Poll

O czym chciał(a)byś się dowiedzieć o potrzebach naszej społeczności?!


# 🙋‍♀️ Event Question

[O czym chciał(a)byś usłyszeć podczas meetupu? Rzuć ciekawym pomysłem na kolejne edycje](https://www.meetup.com/warsaw-data-engineering/events/308350480/attendees/) 🙏

1. ML pipelini in Databrics
1. Tworzenie modeli, dobre praktyki
1. Jak poprawne trenować model
1. Rozpoczęcię swojej przygody z ML

# 📢 News

Things worth watching out for...


## New members in Warsaw Data Engineering!

[You now have 602 members!](https://www.meetup.com/warsaw-data-engineering/members/)

Co zainteresowało Cię w Warsaw Data Engineering Meetup, że zdecydowałaś/-eś się przyłączyć?

1. The Internals Online Books
1. Możliwość zdobycia dodatkowej wiedzy i poznania doświadczeń innych członków.
1. Pogłębić swoje umiejętności



##  Data + AI Summit 2025 is live 🔥

[DAIS](https://www.databricks.com/dataaisummit)


## 🚀 New Versions

What has changed in the tooling space since we last met? I.e. hunting down the features to learn more about.

* [MLflow 2.22.1](https://github.com/mlflow/mlflow/releases/tag/v2.22.1)
    * ⁉️ [3.0.0 RC3 and 3.1.0 RC0](https://github.com/mlflow/mlflow/releases)
* [PydanticAI 0.2.14](https://github.com/pydantic/pydantic-ai/releases/tag/v0.2.14)
    * ⚠️ [Pydantic Graph](https://github.com/pydantic/pydantic-ai/tree/main/pydantic_graph)
    * ⚠️ [Parallel Evaluation](https://ai.pydantic.dev/evals/#parallel-evaluation)
* [Weights & Biases 0.20.1](https://github.com/wandb/wandb/releases/tag/v0.20.1)
    * Konkurencja dla MLflow??? 🤨
* [fast-agent 0.2.29](https://github.com/evalstate/fast-agent/releases/tag/v0.2.29)
* [OpenAI Agents SDK 0.0.17](https://github.com/openai/openai-agents-python/releases/tag/v0.0.17)
* [DSPy: Programming—not prompting—Foundation Models 2.6.26](https://github.com/stanfordnlp/dspy/tree/2.6.26)


# 🧑‍💻 Demo: Model Deployment (Serving, Publishing) in Databricks

Based on [Tutorial: Deploy and query a custom model](https://docs.databricks.com/aws/en/machine-learning/model-serving/model-serving-intro):

1. This article provides the basic steps for deploying and querying a custom model, that is a **traditional ML model**, using **Mosaic AI Model Serving**.
1. The model must be registered in Unity Catalog or in the workspace model registry.

⚠️ Traditional ML models != Generative AI models


There are [3 different logging techniques](https://docs.databricks.com/aws/en/machine-learning/model-serving/model-serving-intro#step-1-log-the-model).

This demo uses manual model logging with [MLflow's built-in model flavors](https://mlflow.org/docs/latest/model#models_built-in-model-flavors) focusing on [scikit-learn (sklearn)](https://mlflow.org/docs/latest/model#scikit-learn-sklearn).

💡 IDEA: Use Custom logging with `pyfunc` for a custom "model" (= some random python code) for extra steps before or after inference.

## Pre-requisites

1. [A Unity Catalog schema to register (save) models to](https://curriculum-dev.cloud.databricks.com/explore/data/jacek_laskowski/mlflow)
1. ⛔️ No Serverless. It did not work for me ⛔️

## Train Model

A scikit-learn [LogisticRegression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) model from [Scikit-learn pyfunc usage](https://mlflow.org/docs/latest/model#scikit-learn-pyfunc-usage)

In [0]:
%pip install mlflow-skinny[databricks]
%restart_python
# dbutils.library.restartPython()

In [0]:
import sys
print(sys.version)

In [0]:
import mlflow
print(mlflow.__version__)

In [0]:
import sklearn
print(sklearn.__version__)

In [0]:
import numpy
print(numpy.__version__)

In [0]:
mlflow.set_registry_uri('databricks-uc')

In [0]:
import mlflow
from mlflow.models import infer_signature
import numpy as np
from sklearn.linear_model import LogisticRegression

with mlflow.start_run():
    X = np.array([-2, -1, 0, 1, 2, 1]).reshape(-1, 1)
    y = np.array([0, 0, 1, 1, 1, 0])
    lr = LogisticRegression()
    lr.fit(X, y)
    signature = infer_signature(X, lr.predict(X))

    mlflow.log_param("wazny_param", 20)
    mlflow.log_param("inny_wazny_param", True)

    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="model",
        signature=signature,
        registered_model_name="jacek_laskowski.mlflow.meetup_06_12",
    )
    print(f"Model saved to: {model_info.model_uri}")


Review [Registered Models](https://curriculum-dev.cloud.databricks.com/ml/models?o=3551974319838082) (Owned by me)

There could be one from my previous experiments.


## List All My Registered Models

<br>

```
databricks registered-models list | jq '.[] | select(.created_by=="jacek@japila.pl")' | jq '.full_name' | tr -d \"
```


## ☠️ Delete All My Registered Models

<br>

```
databricks registered-models list | jq '.[] | select(.created_by=="jacek@japila.pl")' | jq '.full_name' | tr -d \" | while read -r full_model_name;
  do
    echo "Deleting " $full_model_name;
    databricks functions delete --force $full_model_name
  done
```


Review [Model Versions](https://curriculum-dev.cloud.databricks.com/explore/data/models/jacek_laskowski/mlflow/meetup_06_12)

There could be many from my previous experiments.

## Create Serving Endpoint

Based on [Step 2: Create endpoint using the Serving UI](https://docs.databricks.com/aws/en/machine-learning/model-serving/model-serving-intro#step-2-create-endpoint-using-the-serving-ui)

[Create custom model serving endpoints](https://docs.databricks.com/aws/en/machine-learning/model-serving/create-manage-serving-endpoints)


### FIXME Screenshot 🙏

## Query Endpoint

Learn more in [Query serving endpoints for custom models](https://docs.databricks.com/aws/en/machine-learning/model-serving/score-custom-model-endpoints)


### MLflow Client API

In [0]:
import mlflow

model_uri=model_info.model_uri

print(f"model_uri: {model_uri}")

# The always-available generic model loader (pyfunc)
# Could also be mlflow.sklearn.load_model(model_uri=model_uri)
sklearn_pyfunc = mlflow.pyfunc.load_model(model_uri=model_uri)

data = np.array([-4, 1, 0, 10, -2, 1]).reshape(-1, 1)
print(f"data: {data}")

predictions = sklearn_pyfunc.predict(data)
print(f"predictions: {predictions}")

In [0]:
mlflow.sklearn.load_model(model_uri=model_uri)

### Databricks UI

1. [Serving endpoints](https://curriculum-dev.cloud.databricks.com/ml/endpoints) > Click **Owned by me** to narrow down the list of the available endpoints > [jacek_laskowski_meetup_06_12](https://curriculum-dev.cloud.databricks.com/ml/endpoints/jacek_laskowski_meetup_06_12)
1. Click **Use** (in the top-right corner) or **Query** (in the drop-down menu) buttons
1. In **Browser** tab, enter one of the [supported scoring formats](https://docs.databricks.com/aws/en/machine-learning/model-serving/score-custom-model-endpoints#supported-scoring-formats):
    * [JSONified Pandas DataFrame](https://docs.databricks.com/aws/en/machine-learning/model-serving/score-custom-model-endpoints#pandas-dataframe)
    * [Tensor input](https://docs.databricks.com/aws/en/machine-learning/model-serving/score-custom-model-endpoints#tensor-input) (e.g., `{"inputs": [[-4], [ 1], [ 0], [10], [-2], [ 1]]}`)


### Command Line

<br>

```
❯ echo -n '{"inputs": [[5]]}' | https -A bearer -a $DATABRICKS_TOKEN https://curriculum-dev.cloud.databricks.com/serving-endpoints/jacek_laskowski_meetup_06_12_v1/invocations
HTTP/1.1 200 OK
content-length: 20
content-type: application/json
date: Sun, 8 Jun 2025 09:35:21 GMT
endpoint-scale-to-zero: True
endpoint-small-workload: True
served-model-name: meetup_06_12-1
server: databricks
strict-transport-security: max-age=31536000; includeSubDomains; preload
x-content-type-options: nosniff
x-databricks-org-id: 3551974319838082
x-request-id: f961f144-681e-4449-9cd4-c2e295d87f52

{
    "predictions": [
        1
    ]
}
```


You can learn quite a lot from errors (when input data is of incorrect format):

<br>

```
❯ echo -n '{}' | https -A bearer -a $DATABRICKS_TOKEN https://curriculum-dev.cloud.databricks.com/serving-endpoints/jacek_laskowski_meetup_06_12_v1/invocations
HTTP/1.1 400 Bad Request
content-encoding: gzip
content-type: application/json
date: Sun, 8 Jun 2025 09:38:37 GMT
endpoint-scale-to-zero: True
endpoint-small-workload: True
served-model-name: meetup_06_12-1
server: databricks
strict-transport-security: max-age=31536000; includeSubDomains; preload
transfer-encoding: chunked
vary: Accept-Encoding
x-content-type-options: nosniff
x-databricks-org-id: 3551974319838082
x-request-id: 61b79acd-4484-4e23-ba5b-aa868017bba2

{
    "error_code": "BAD_REQUEST",
    "message": "Invalid input. The input must be a JSON dictionary with exactly one of the input fields {'inputs', 'instances', 'dataframe_split', 'dataframe_records'}.. Received dictionary with input fields: set()."
}
```

The input must be a JSON dictionary with exactly one of the input fields `{'inputs', 'instances', 'dataframe_split', 'dataframe_records'}`


## 🎁 Bonus: Databricks CLI


### databricks serving-endpoints

The [Serving Endpoints API](https://docs.databricks.com/api/workspace/servingendpoints) allows you to create, update, and delete model serving endpoints.

1. You can use a **serving endpoint** to serve models from the Databricks Model Registry or Unity Catalog.
1. Endpoints expose the underlying models as scalable REST API endpoints using serverless compute.
    1. The endpoints and associated compute resources are fully managed by Databricks and will not appear in your cloud account.
1. A serving endpoint can consist of one or more MLflow models from the Databricks Model Registry, called **served entities**.
    1. A serving endpoint can have at most ten served entities.
    1. You can configure traffic settings to define how requests should be routed to your served entities behind an endpoint.
    1. Additionally, you can configure the scale of resources that should be applied to each served entity.

Available commands in `databricks serving-endpoints`:

| Command | Description |
|-|-|
| build-logs                                    | Get build logs for a served model. |
| create                                        | Create a new serving endpoint. |
| create-provisioned-throughput-endpoint        | Create a new PT serving endpoint. |
| delete                                        | Delete a serving endpoint. |
| export-metrics                                | Get metrics of a serving endpoint. |
| get                                           | Get a single serving endpoint. |
| get-open-api                                  | Get the schema for a serving endpoint. |
| list                                          | Get all serving endpoints. |
| logs                                          | Get the latest logs for a served model. |
| patch                                         | Update tags of a serving endpoint. |
| put                                           | Update rate limits of a serving endpoint. |
| put-ai-gateway                                | Update AI Gateway of a serving endpoint. |
| query                                         | Query a serving endpoint. |
| update-config                                 | Update config of a serving endpoint. |
| update-provisioned-throughput-endpoint-config | Update config of a PT serving endpoint. |

The Permission Commands in `databricks serving-endpoints`:

| Permission Command | Description |
|-|-|
| get-permission-levels                         | Get serving endpoint permission levels. |
| get-permissions                               | Get serving endpoint permissions. |
| set-permissions                               | Set serving endpoint permissions. |
| update-permissions                            | Update serving endpoint permissions. |


### List My Serving Endpoints

<br>

```
databricks serving-endpoints list | jq '.[] | select(.creator=="jacek@japila.pl")'
```


### List Names of My Serving Endpoints

<br>

```
databricks serving-endpoints list | jq '.[] | select(.creator=="jacek@japila.pl")' | jq '.name'
```



Once up and running, get the query schema of the serving endpoint in OpenAPI format.

<br>

```
databricks serving-endpoints get-open-api jacek_laskowski_meetup_06_12
```


### ☠️ Delete All My Endpoints

<br>

```
models=$(databricks serving-endpoints list | jq '.[] | select(.creator=="jacek@japila.pl")' | jq '.name')
echo $models | tr -d \" | while read -r model_name;
  do
    if [ $model_name = 'DISABLED_jacek_laskowski_meetup_06_12' ];
    then
      echo "Skip " $model_name;
    else
      echo "Deleting " $model_name;
      databricks serving-endpoints delete $model_name
    fi
  done
```


## 🛑 Stop Here

In [0]:
dbutils.notebook.exit("Stop here")


# 🧑‍💻 Demo: Running MLflow's examples/databricks (and Editable Install in Python)

[examples/databricks](https://github.com/mlflow/mlflow/tree/master/examples/databricks)


## Step 0. Clone MLflow Repo

`git clone` https://github.com/mlflow/mlflow


## Step 1. Install Dependencies


```
uv pip install databricks-connect
uv pip install scikit-learn
```


## Step 2. Run Experiment


```
❯ python examples/databricks/dbconnect.py --cluster-id xxx
2025/05/08 17:51:04 INFO mlflow.tracking.fluent: Experiment with name '/Users/jacek@japila.pl/dbconnect' does not exist. Creating a new experiment.
🏃 View run smiling-ox-667 at: https://curriculum-dev.cloud.databricks.com/ml/experiments/1275781889574864/runs/b88fd8406e7d410bac8992258093ef5d
🧪 View experiment at: https://curriculum-dev.cloud.databricks.com/ml/experiments/1275781889574864
Traceback (most recent call last):
  File "/Users/jacek/oss/mlflow/examples/databricks/dbconnect.py", line 56, in <module>
    main()
    ~~~~^^
  File "/Users/jacek/oss/mlflow/examples/databricks/dbconnect.py", line 37, in main
    model_info = mlflow.sklearn.log_model(model, name="model", signature=signature)
TypeError: log_model() got an unexpected keyword argument 'name'
```


## Step 3. Editable Install

[Development Mode (a.k.a. “Editable Installs”)](https://setuptools.pypa.io/en/latest/userguide/development_mode.html)


```
uv pip install -e .
```


```
❯ python examples/databricks/dbconnect.py --cluster-id xxx
🏃 View run carefree-duck-680 at: https://curriculum-dev.cloud.databricks.com/ml/experiments/1275781889574864/runs/89a774fb54da4a5c844764d3e40ad638
🧪 View experiment at: https://curriculum-dev.cloud.databricks.com/ml/experiments/1275781889574864
Traceback (most recent call last):
  File "/Users/jacek/oss/mlflow/examples/databricks/dbconnect.py", line 56, in <module>
    main()
    ~~~~^^
  File "/Users/jacek/oss/mlflow/examples/databricks/dbconnect.py", line 37, in main
    model_info = mlflow.sklearn.log_model(model, name="model", signature=signature)
  File "/Users/jacek/oss/mlflow/mlflow/sklearn/__init__.py", line 426, in log_model
    return Model.log(
           ~~~~~~~~~^
        artifact_path=artifact_path,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<18 lines>...
        model_id=model_id,
        ^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/jacek/oss/mlflow/mlflow/models/model.py", line 928, in log
    model = mlflow.initialize_logged_model(
        # TODO: Update model name
    ...<6 lines>...
        else None,
    )
  File "/Users/jacek/oss/mlflow/mlflow/tracking/fluent.py", line 2122, in initialize_logged_model
    model = _create_logged_model(
        name=name,
    ...<4 lines>...
        experiment_id=experiment_id,
    )
  File "/Users/jacek/oss/mlflow/mlflow/tracking/fluent.py", line 2232, in _create_logged_model
    return MlflowClient().create_logged_model(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        experiment_id=experiment_id,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<4 lines>...
        model_type=model_type,
        ^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/jacek/oss/mlflow/mlflow/tracking/client.py", line 5218, in create_logged_model
    return self._tracking_client.create_logged_model(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        experiment_id, name, source_run_id, tags, params, model_type
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/jacek/oss/mlflow/mlflow/tracking/_tracking_service/client.py", line 815, in create_logged_model
    return self.store.create_logged_model(
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        experiment_id=experiment_id,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    ...<8 lines>...
        model_type=model_type,
        ^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/jacek/oss/mlflow/mlflow/store/tracking/rest_store.py", line 904, in create_logged_model
    response_proto = self._call_endpoint(CreateLoggedModel, req_body)
  File "/Users/jacek/oss/mlflow/mlflow/store/tracking/rest_store.py", line 129, in _call_endpoint
    return call_endpoint(
        self.get_host_creds(),
    ...<4 lines>...
        retry_timeout_seconds=retry_timeout_seconds,
    )
  File "/Users/jacek/oss/mlflow/mlflow/utils/rest_utils.py", line 474, in call_endpoint
    response = verify_rest_response(response, endpoint)
  File "/Users/jacek/oss/mlflow/mlflow/utils/rest_utils.py", line 261, in verify_rest_response
    raise RestException(json.loads(response.text))
mlflow.exceptions.RestException: BAD_REQUEST: This API is not enabled.
```

## Step 4. BAD_REQUEST: This API is not enabled.

Hunting down the root cause of the exception.


Modify `mlflow/utils/rest_utils.py:261`


```
❯ python examples/databricks/dbconnect.py --cluster-id xxx
>>> endpoint /api/2.0/mlflow/experiments/get-by-name
>>> endpoint /api/2.0/mlflow/runs/create
>>> endpoint /api/2.0/mlflow/runs/get
>>> endpoint /api/2.0/mlflow/logged-models
>>> endpoint /api/2.0/mlflow/runs/get
🏃 View run zealous-worm-360 at: https://curriculum-dev.cloud.databricks.com/ml/experiments/1275781889574864/runs/8cad690b9bdd45ab96658987f4039180
🧪 View experiment at: https://curriculum-dev.cloud.databricks.com/ml/experiments/1275781889574864
>>> endpoint /api/2.0/mlflow/runs/update
Traceback (most recent call last):
  File "/Users/jacek/oss/mlflow/examples/databricks/dbconnect.py", line 56, in <module>
    main()
    ~~~~^^
...
  File "/Users/jacek/oss/mlflow/mlflow/utils/rest_utils.py", line 262, in verify_rest_response
    raise RestException(json.loads(response.text))
mlflow.exceptions.RestException: BAD_REQUEST: This API is not enabled.
```


## Step 5. MLflow API reference

[MLflow API reference](https://docs.databricks.com/aws/en/reference/mlflow-api)


### Experiments

[Experiments](https://docs.databricks.com/api/workspace/experiments)

1. **Experiments** are the primary unit of organization in MLflow.
1. All **MLflow runs** belong to an experiment.
1. Each experiment lets you visualize, search, and compare runs, as well as download run artifacts or metadata for analysis in other tools.
1. Experiments are maintained in a Databricks-hosted MLflow tracking server.
1. Experiments are located in the workspace file tree.
1. You manage experiments using the same tools you use to manage other workspace objects such as folders, notebooks, and libraries.

### Databricks CLI

<br>

```
❯ databricks | more
...
Machine Learning
  experiments                            Experiments are the primary unit of organization in MLflow; all MLflow runs belong to an experiment.
  model-registry                         Note: This API reference documents APIs for the Workspace Model Registry.
Real-time Serving
  serving-endpoints                      The Serving Endpoints API allows you to create, update, and delete model serving endpoints.
Unity Catalog
  model-versions                         Databricks provides a hosted version of MLflow Model Registry in Unity Catalog.
  registered-models                      Databricks provides a hosted version of MLflow Model Registry in Unity Catalog.
...
```


```
❯ databricks experiments list-experiments | jq '.[].name' | grep 'jacek@japila.pl'
"/Users/jacek@japila.pl/dbconnect"
"/Users/jacek@japila.pl/demo-experiment"
```


```
databricks registered-models list | jq '.[].full_name'
```

# 🧑‍💻 Demo: MLflow's dev/pyproject.py and uv

1. What I learnt while reviewing the source code of MLflow and having found [dev/pyproject.py](https://github.com/mlflow/mlflow/blob/master/dev/pyproject.py) to execute locally.
1. And how uv helped.

Why it even matters?! 🤨


## Step 0. Clone MLflow Repo

`git clone` https://github.com/mlflow/mlflow


## Step 1. uvx python dev/pyproject.py

<br>

```
❯ uvx python dev/pyproject.py
Traceback (most recent call last):
  File "/Users/jacek/oss/mlflow/./dev/pyproject.py", line 10, in <module>
    import toml
ModuleNotFoundError: No module named 'toml'
```


## Step 2. Set Up Dev Env


`uv venv .dev_pyproject_py_deep_dive`

`source .dev_pyproject_py_deep_dive/bin/activate`


## Step 3. Virtual Envs in Python

Please note that I'm a JVM dev (and only very recently switched to Python).


`uv pip install toml`

`python ./dev/pyproject.py`

`type python` and it finally clicked how virtual envs work 🔥

[venv — Creation of virtual environments](https://docs.python.org/3/library/venv.html)

## Step 4. It Works 🥳


`uv pip install pyyaml`

> ⚠️ NOTE
>
> All the dev deps are in [dev/requirements.txt](https://github.com/mlflow/mlflow/blob/master/dev/requirements.txt)

`uv pip install packaging`

`brew install taplo`

`python ./dev/pyproject.py` seems to change nothing, huh?! 🤨

💎 Think what the script does and you will know why nothing seems changed 😉

# ✨ Bonus Demo ✨

[RestException: INVALID_PARAMETER_VALUE while searching for registered models from model registry](https://stackoverflow.com/q/79630371/1305344)

# That's all Folks! 👋

![Warner Bros., Public domain, via Wikimedia Commons](https://upload.wikimedia.org/wikipedia/commons/e/ea/Thats_all_folks.svg)


# 🙋 Questions and Answers


# 💡 Ideas for Future Events

➡️ [Ideas for Future Events]($./Ideas for Future Events)